In [12]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
import tensorflow.keras
from sklearn.preprocessing import LabelBinarizer

from sklearn.model_selection import  train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib as plt 
import numpy as np
import os 


In [5]:
init_lr = 1e-4 
epochs = 20 
BS = 32 
dictory = r'D:\engineer\AMIT\Face-Mask-Detection-master\dataset'
categories = ['with_mask' , 'without_mask']

data = []
labels = []


for category in categories:
    path = os.path.join(dictory , category)
    for img in os.listdir(path):
        img_path = os.path.join(path,img)
        image = load_img(img_path , target_size = (224,224))
        image = img_to_array(image)
        image = preprocess_input(image)
        
        data.append(image)
        labels.append(category)

C:\Users\PC\anaconda3\lib\site-packages\PIL\Image.py:959: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


In [8]:
#perform one-hot encdoing on the labels 
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

data = np.array(data, dtype='float32')
labels= np.array(labels)

trainX , testX , trainY , testY = train_test_split(data , labels, 
                test_size = 0.20 , stratify = labels , random_state = 42 )

In [9]:
# augmentaion
aug = ImageDataGenerator(
    rotation_range=20 , 
    zoom_range= 0.15 , 
    width_shift_range= .2, 
    shear_range= .15 ,
    horizontal_flip=True,
    fill_mode='nearest'  )

In [16]:
# load the mobileNetV2
baseModel = MobileNetV2(weights='imagenet', include_top=False, 
                       input_tensor=Input(shape=(224,224,3)))
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7,7))(headModel)
headModel = Flatten(name = 'flatten')(headModel)
headModel = Dense(128 , activation='relu')(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2 , activation='softmax')(headModel)

model = Model(inputs=baseModel.input , outputs = headModel)


In [19]:
# loop over all layers 
for layer in baseModel.layers :
    layer.trainable = False
    
# compile the model 

opt = Adam(learning_rate=init_lr )
model.compile(loss='binary_crossentropy' , optimizer = opt , 
             metrics = ['accuracy'])

# train the head of the network 

H = model.fit(
    aug.flow(trainX ,trainY , batch_size=BS), 
    steps_per_epoch= len(trainX)//BS, 
    validation_data=(testX ,testY ),
    validation_steps= len(testX)//BS, 
    epochs = epochs
                )

Epoch 1/20
95/95 [==============================] - 55s 553ms/step - loss: 0.4068 - accuracy: 0.8649 - val_loss: 0.1564 - val_accuracy: 0.9857
Epoch 2/20
95/95 [==============================] - 43s 456ms/step - loss: 0.1626 - accuracy: 0.9621 - val_loss: 0.0834 - val_accuracy: 0.9883
Epoch 3/20
95/95 [==============================] - 42s 439ms/step - loss: 0.1079 - accuracy: 0.9717 - val_loss: 0.0571 - val_accuracy: 0.9922
Epoch 4/20
95/95 [==============================] - 41s 433ms/step - loss: 0.0822 - accuracy: 0.9796 - val_loss: 0.0492 - val_accuracy: 0.9922
Epoch 5/20
95/95 [==============================] - 39s 407ms/step - loss: 0.0590 - accuracy: 0.9845 - val_loss: 0.0386 - val_accuracy: 0.9922
Epoch 6/20
95/95 [==============================] - 44s 458ms/step - loss: 0.0532 - accuracy: 0.9855 - val_loss: 0.0353 - val_accuracy: 0.9935
Epoch 7/20
95/95 [==============================] - 85s 896ms/step - loss: 0.0588 - accuracy: 0.9822 - val_loss: 0.0329 - val_accuracy: 0.9922

In [22]:
# make prediction 
predIdxs = model.predict(testX , batch_size= BS)

predIdxs = np.argmax(predIdxs , axis = 1 )

24/24 [==============================] - 6s 255ms/step


In [23]:
# show classification model
print(classification_report(testY.argmax(axis = 1) , predIdxs , 
                           target_names=lb.classes_))

              precision    recall  f1-score   support

   with_mask       0.99      0.99      0.99       383
without_mask       0.99      0.99      0.99       384

    accuracy                           0.99       767
   macro avg       0.99      0.99      0.99       767
weighted avg       0.99      0.99      0.99       767



In [26]:
print('saving mask detection model')
model.save("mask_detection_model_2" , save_format='h5')

saving mask detection model
